In [0]:
import numpy as np
import matplotlib.pyplot as plt
import random
np.random.seed(1)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
DATA_PATH = r'/content/gdrive/My Drive/app/MNIST'
X_train = np.load(DATA_PATH + '/x_train.npy')
y_train = one_hot(np.load(DATA_PATH + '/y_train.npy'),10)
X_val   = np.load(DATA_PATH + '/x_val.npy')
y_val   = one_hot(np.load(DATA_PATH + '/y_val.npy'),10)
X_test  = np.load(DATA_PATH + '/x_test.npy')
y_test  = one_hot(np.load(DATA_PATH + '/y_test.npy'),10)

In [0]:
def one_hot(labels, n):
    m = len(labels)
    onehot = np.zeros((m, n))
    onehot[np.arange(m), labels] = 1
    return onehot

def accuracy(y_pred, y):
    return np.sum(np.argmax(y_pred, axis=1) == np.argmax(y, axis=1)) / y.shape[0]


def data_iter(data, batch_size):
    X, y = data
    batches = [(X[i:i+batch_size],y[i:i+batch_size]) for i in range(0,X.shape[0],batch_size)]
    random.shuffle(batches)
    for batch in batches:
        yield batch
                
        
def glorot(in_dim, out_dim):
    d = np.sqrt(6/(in_dim+out_dim))
    return np.random.normal(-d,d,(in_dim,out_dim))

In [0]:
INPUT_DIM = 784
OUTPUT_DIM = 10

class NN(object):
    
    
    def __init__(self,hidden_dims=(1024,2048),n_hidden=2,init_method='normal',activate='relu',mode='train'):
        self.dims = [INPUT_DIM,] + hidden_dims + [OUTPUT_DIM,]
        self.weights = []
        self.biases  = []
        self.init = init_method
        self.activate = activate
        
        initialize_weights(n_hidden, self.dims)
        
        
    def initialize_weights(self, n_hidden, dims):
        if self.init == 'Zero':
            init_method = lambda x, y: np.zeros(x,y)
        elif self.init == 'Normal':
            init_method = lambda x, y: np.random.randn(x,y)
        elif self.init == 'Glorot':
            init_method = glorot
            
        for (inputs, outputs) in zip(dims[:-1], dims[1:]):
            self.weights.append(init_method(inputs, outputs))
            self.biases.append(np.zeros(outputs))
            
            
    def activation(self,inputs):
        if self.activate == 'relu':
            return np.max(inputs,0)
        if self.activate == 'sigmoid':
            return 1.0/(1.0+np.exp(-inputs))
            
            
    def forward(self, inputs):
        a_k = None
        h_k = inputs
        a = []
        h = []
        for (W, b) in zip(self.weights[:-1], self.biases[:-1]):
            a_k = np.dot(h_k, W)
            h_k = activation(a_k)
            a.append(a_k)
            h.append(h_k)
        
        a_k = np.dot(h_k, self.weights[-1]) + self.biases[-1]
        h_k = softmax(a_k)
        a.append(a_k)
        h.append(h_k)
        cache = {'a':a, 'h':h}
        
        return h_k, cache
    
    
    
    def loss(self, pred, labels):
        '''
        Negative log likelihood
        '''
        ls = np.log(pred)
        ls = - np.sum(ls * labels)
        return ls
    
    
    def backward(self,cache,labels,...):
        
        
        
    def update(self,grads,lr):
        grads_w = grads['w']
        grads_b = grads['b']
        for i in range(len(self.weights)):
            self.weights[i] -= lr * grads_w[i]
            self.biases[i] -= lr * grads_b[i]
            

    def train(self, data, epochs, batch_size, lr, lambd=0.0, test_data=None):
        for ep in range(1, epochs+1):
            for (batch_x, batch_y) in data_iter(data, batch_size):
                
                y_pred, cache = self.forward(batch_x, batch_y)
                grads = self.backward(cache)
                self.update(grads, lr)

                
    def test(self, data):
        x, y = data
        outputs, _ = forward(x)
        outputs = onehot_reverse(outputs)
        acc = np.sum(outputs == y) / len(data)
        
    
    def activate_grad(inputs):
        if self.activate == 'relu':
            inputs[inputs > 0] = 1
            inputs[inputs < 0] = 0
        elif self.activate == 'sigmiod':
            return activation(inputs) * (1 - activation(inputs))
        
        
    def softmax(self,inputs):
        outputs = np.exp(inputs)
        return outputs / (np.sum(outputs, axis=1))
        

3

In [0]:
a = np.arange(12)
a[a > 1] = 1
a

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])